## DEBUG: this should be moved outside

In [3]:
Progress <- R6::R6Class("Progress",
  inherit = KerasCallback,
  
  public = list(
    num_epochs  = NULL,
    update_frequency = NULL,
    epoch = NULL,
    batch = NULL,
      
    initialize = function(num_epochs, update_frequency) {
        self$num_epochs <- num_epochs
        self$update_frequency <- update_frequency
        self$epoch <- 1
    },
                        
    on_batch_end = function(batch, logs = list()) {
        if ((batch + 1) %% self$update_frequency == 0) {
            cat('Epoch ', self$epoch + 1, '/', self$num_epochs, ': ', batch + 1,
                ' - loss: ', logs[['loss']], '\r', sep = '')
            flush.console()
        }
        self$batch <- batch
    },
      
    on_epoch_begin = function(epoch, logs = list()) {
        self$epoch <- epoch
    },
                        
    on_epoch_end = function(epoch, logs = list()) {
        cat('Epoch ', self$epoch + 1, '/', self$num_epochs, ': ', self$batch + 1, ' - loss: ', logs[['loss']], ' - validation loss: ', logs[['val_loss']], '\n', sep = '')
        flush.console()
    }
))

# Classification
In this notebook, we will revisit the Boston housing data set from the last session. It is quite difficult to build a model that performs highly accurate regression. Instead of increasing our model complexity, we can turn the regression problem into a *classification* problem. That way, we do not predict the housing price directly, but predict to a number of *classes* or *labels* that contain certain price ranges.

You could argue that we are relaxing our standards a bit in this manner, but it is often a valid approach towards 'solving' a difficult regression problem.

In [1]:
library(keras)
library(ggplot2)
library(tidyverse)
library(rsample)
library(reshape2)

library(repr)
options(repr.plot.width=6, repr.plot.height=4)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.0.1     ✔ purrr   0.2.5
✔ tidyr   0.8.2     ✔ dplyr   0.7.8
✔ readr   1.3.1     ✔ stringr 1.3.1
✔ tibble  2.0.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths



In [ ]:
data <- dataset_boston_housing()

In [ ]:
c(c(X_train, y_train), c(X_test, y_test)) %<-% data
dim(X_train)

In [ ]:
hist(X_train[1:dim(X_train)[1], 13])

In [ ]:
hist(y_train)

In [ ]:
correlation <- round(cor(X_train), 2)
correlation

In [ ]:
ggplot(data = melt(correlation), aes(x = Var1, y = Var2, fill = value)) + geom_tile()

In [ ]:
# y_train <- y_train %>% cut(breaks = c(1, 25, max(y_train)), labels = FALSE) - 1

In [ ]:
breaks <- c(0, 10, 20, 30, 40, max(y_train))
y_train_multiclass <- y_train %>% cut(breaks = breaks, labels = FALSE) - 1
table(y_train_multiclass)

In [ ]:
1 / table(y_train_multiclass)

In [ ]:
frequencies <- table(y_train_multiclass)
class_weights <- vector(mode = "list", length = length(frequencies))
names(class_weights) <- as.character(seq(0, length(frequencies) - 1))
for (i in 1:length(names(class_weights))) {
    key <- names(class_weights)[[i]]
    class_weights[[key]] <- 1 / frequencies[[key]]
}

In [ ]:
y_train <- to_categorical(y_train_multiclass)
dim(y_train)

In [ ]:
dim(y_train)

## Normalize the data
Normalize the data as before, by centering on the mean of the training data, and dividing all values by their standard deviation.

In [ ]:
X_mean <- apply(X_train, 2, mean)
X_sd <- apply(X_train, 2, sd)

In [ ]:
X_train <- scale(X_train, center = X_mean, scale = X_sd)
X_test  <- scale(X_test, center = X_mean, scale = X_sd)

## Split data

In [ ]:
split_at <- 0.8 * dim(X_train)[1]
X_val <- X_train[-(1:split_at),]
y_val <- y_train[-(1:split_at),]
X_train <- X_train[1:split_at,]
y_train <- y_train[1:split_at,]

In [ ]:
dim(y_val)

In [ ]:
model <- keras_model_sequential() %>%
    layer_dense(units = 16, input_shape=dim(X_train)[2], activation = "sigmoid") %>%
#     layer_dense(units = 256, activation = "sigmoid") %>%
    layer_dense(units = 5, activation = "softmax")

model %>% compile(
    optimizer = optimizer_rmsprop(lr = 0.001),
    loss = "categorical_crossentropy",
    metrics = c("accuracy")
)
cat(summary(model))
num_epochs <- 250
history <- model %>% fit(
    X_train, y_train,
    validation_data = list(X_val, y_val),
    epochs = num_epochs,
    batch_size = 16,
    callbacks=list(Progress$new(num_epochs, 10))
)
plot(history)

In [ ]:
y_pred <- model %>% predict(X_val)
table(apply(y_pred, 1, which.max))

In [ ]:
model <- keras_model_sequential() %>%
    layer_dense(units = 16, input_shape=dim(X_train)[2], activation = "sigmoid") %>%
#     layer_dense(units = 256, activation = "sigmoid") %>%
    layer_dense(units = 5, activation = "softmax")

model %>% compile(
    optimizer = optimizer_rmsprop(lr = 0.001),
    loss = "categorical_crossentropy",
    metrics = c("accuracy")
)
cat(summary(model))
num_epochs <- 250
history <- model %>% fit(
    X_train, y_train,
    validation_data = list(X_val, y_val),
    epochs = num_epochs,
    batch_size = 16,
    callbacks = list(Progress$new(num_epochs, 10)),
    class_weights = class_weights
)
plot(history)

In [ ]:
y_pred <- model %>% predict(X_val)
table(apply(y_pred, 1, which.max))

## DEBUG: airplane data set

In [5]:
data <- read.csv('airplane/2018/airplane-delay-2018-1.csv')
glimpse(data)

Observations: 570,118
Variables: 33
$ YEAR                <int> 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018, 2…
$ MONTH               <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ DAY_OF_MONTH        <int> 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2…
$ DAY_OF_WEEK         <int> 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6…
$ FL_DATE             <fct> 2018-01-27, 2018-01-27, 2018-01-27, 2018-01-27, 2…
$ OP_UNIQUE_CARRIER   <fct> UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, UA, U…
$ TAIL_NUM            <fct> N26232, N477UA, N13720, N16217, N33714, N809UA, N…
$ OP_CARRIER_FL_NUM   <int> 369, 368, 367, 366, 365, 364, 363, 362, 361, 360,…
$ ORIGIN              <fct> FLL, SEA, DCA, LAX, JAX, IAH, EWR, HNL, LAS, IAD,…
$ ORIGIN_CITY_NAME    <fct> "Fort Lauderdale, FL", "Seattle, WA", "Washington…
$ DEST                <fct> IAH, SFO, IAH, ORD, EWR, PHX, HNL, EWR, SFO, TPA,…
$ DEST_CITY_NAME      <fct> "Houston, TX", "San Francisco, CA", "Houston, TX"…
$ CRS_DEP_TIME  

In [6]:
subset <- data[c('DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'DEP_DELAY', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'DISTANCE')]
head(subset)

DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST,DEP_DELAY,CRS_DEP_TIME,CRS_ARR_TIME,DISTANCE
27,6,UA,N26232,369,FLL,IAH,-13,615,808,966
27,6,UA,N477UA,368,SEA,SFO,-4,618,831,679
27,6,UA,N13720,367,DCA,IAH,-2,830,1107,1208
27,6,UA,N16217,366,LAX,ORD,-9,650,1250,1744
27,6,UA,N33714,365,JAX,EWR,-14,1824,2045,820
27,6,UA,N809UA,364,IAH,PHX,-7,1420,1618,1009


In [7]:
subset$OP_UNIQUE_CARRIER <- as.numeric(subset$OP_UNIQUE_CARRIER)
subset$TAIL_NUM <- as.numeric(subset$TAIL_NUM)
subset$OP_CARRIER_FL_NUM <- as.numeric(subset$OP_CARRIER_FL_NUM)
subset$ORIGIN <- as.numeric(subset$ORIGIN)
subset$DEST <- as.numeric(subset$DEST)

In [8]:
glimpse(subset)

Observations: 570,118
Variables: 11
$ DAY_OF_MONTH      <int> 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,…
$ DAY_OF_WEEK       <int> 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, …
$ OP_UNIQUE_CARRIER <dbl> 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14,…
$ TAIL_NUM          <dbl> 913, 1846, 275, 417, 1257, 3589, 2324, 2324, 1396, …
$ OP_CARRIER_FL_NUM <dbl> 369, 368, 367, 366, 365, 364, 363, 362, 361, 360, 3…
$ ORIGIN            <dbl> 113, 287, 85, 176, 165, 151, 105, 140, 174, 149, 17…
$ DEST              <dbl> 151, 289, 151, 231, 105, 244, 140, 105, 289, 316, 1…
$ DEP_DELAY         <dbl> -13, -4, -2, -9, -14, -7, 27, 8, -5, -7, -6, -9, -8…
$ CRS_DEP_TIME      <int> 615, 618, 830, 650, 1824, 1420, 815, 1615, 1520, 17…
$ CRS_ARR_TIME      <int> 808, 831, 1107, 1250, 2045, 1618, 1438, 653, 1703, …
$ DISTANCE          <dbl> 966, 679, 1208, 1744, 820, 1009, 4962, 4962, 414, 8…


In [9]:
X_train <- as.matrix(subset[c('DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_UNIQUE_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'CRS_ARR_TIME', 'DISTANCE')], replace = TRUE)
dimnames(X_train) <- NULL
dim(X_train)

[1] 570118     10

In [14]:
y_train <- subset$DEP_DELAY
# y_train[y_train < -50] = -50
# y_train[y_train > 50] = 50
y_train[is.na(y_train)] = 0
y_train <- as.numeric(y_train > 60)
table(y_train)

y_train
     0      1 
537354  32764 

In [20]:
inverse_frequencies <- 1 / table(y_train)
class_weights <- inverse_frequencies / sum(inverse_frequencies)
class_weights

y_train
        0         1 
0.0574688 0.9425312 

In [21]:
y_train <- to_categorical(y_train)
head(y_train)

1,0
1,0
1,0
1,0
1,0
1,0


In [22]:
split_at <- 0.8 * dim(X_train)[1]
X_val <- X_train[-(1:split_at),]
y_val <- y_train[-(1:split_at),]
X_train <- X_train[1:split_at,]
y_train <- y_train[1:split_at,]

In [23]:
dim(X_val)

[1] 114024     10

In [24]:
dim(y_val)

[1] 114024      2

In [25]:
head(X_train)

27,6,14,913,369,113,151,615,808,966
27,6,14,1846,368,287,289,618,831,679
27,6,14,275,367,85,151,830,1107,1208
27,6,14,417,366,176,231,650,1250,1744
27,6,14,1257,365,165,105,1824,2045,820
27,6,14,3589,364,151,244,1420,1618,1009


In [4]:
model <- keras_model_sequential() %>%
#     layer_dense(units = 32, input_shape=dim(X_train)[2], activation = "relu") %>%
#     layer_dense(units = 256, activation = "sigmoid") %>%
    layer_dense(units = 2, , input_shape=dim(X_train)[2], activation = "softmax")

model %>% compile(
    optimizer = optimizer_rmsprop(lr = 0.001),
    loss = "binary_crossentropy",
    metrics = c("accuracy")
)
cat(summary(model))

num_epochs <- 10
history <- model %>% fit(
    X_train, y_train,
    validation_data = list(X_val, y_val),
    epochs = num_epochs,
    batch_size = 1024,
    callbacks = list(Progress$new(num_epochs, 10)),
    shuffle = TRUE,
    class_weight = list(class_weights)
)
plot(history)

ERROR: Error in normalize_shape(input_shape): object 'X_train' not found


In [42]:
head(apply(model %>% predict(X_val), 1, which.max) - 1)

[1] 0 0 0 0 0 0